# Demographics Processing

Process the demographics data from https://www.cde.ca.gov/ds/ad/filesenrcensus.asp

In [1]:
%run notebooks/Setup.ipynb

import json
import polars as polars

## Pivot Data by Race and Ethnicity

In [12]:
# load all the data
all_data = polars.read_csv(workspace_path.joinpath('data/raw/dataquest/cdenroll2324-v2.txt'), separator='\t', ignore_errors=True)

# only look at San Francisco Unified at the school level
sf_unified = all_data.filter(
    (polars.col('DistrictName') == 'San Francisco Unified') & 
    (polars.col('AggregateLevel') == 'S')
)

# filter for rows reporting on race and ethnicity
sf_unified = sf_unified.filter(
    polars.col('ReportingCategory').str.starts_with('RE_')
)

# count the number of schools in the dataset
sf_unified['SchoolName'].n_unique()

119

In [13]:
# defined codes
reporting_category_map = {
    'RE_A': 'Asian',
    'RE_B': 'African American',
    'RE_D': 'Not Reported',
    'RE_F': 'Filipino',
    'RE_H': 'Hispanic or Latino',
    'RE_I': 'American Indian or Alaska Native',
    'RE_P': 'Pacific Islander',
    'RE_T': 'Two or More Races',
    'RE_W': 'White'
}

# map from codes to labels
sf_unified = sf_unified.with_columns(
    polars.col('ReportingCategory')
        .map_elements(lambda x: reporting_category_map.get(x, x), return_dtype=polars.String)
        .alias('Race/Ethnicity')
)

# Rename TOTAL_ENR to GR_ALL since adding more totals will make it confusing
sf_unified = sf_unified.rename({'TOTAL_ENR': 'GR_ALL'})

# clean up some columns
sf_unified = sf_unified.drop([
    'AcademicYear', 'AggregateLevel', 'CountyCode', 'DistrictCode', 'SchoolCode', 'CountyName', 'DistrictName', 'ReportingCategory'
])

sf_unified.head()

SchoolName,Charter,GR_ALL,GR_TK,GR_KN,GR_01,GR_02,GR_03,GR_04,GR_05,GR_06,GR_07,GR_08,GR_09,GR_10,GR_11,GR_12,Race/Ethnicity
str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str
"""District Office""","""N""",21,21,0,0,0,0,0,0,0,0,0,0,0,0,0,"""Asian"""
"""District Office""","""N""",1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,"""African American"""
"""District Office""","""N""",2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,"""Not Reported"""
"""District Office""","""N""",2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,"""Filipino"""
"""District Office""","""N""",21,21,0,0,0,0,0,0,0,0,0,0,0,0,0,"""Hispanic or Latino"""


In [14]:
# compute total enrolled across all race/ethnicities per school
total_enrolled = sf_unified.group_by('SchoolName').agg(
    TOTAL_ALL=polars.col('GR_ALL').sum()
)

# add the TOTAL_ALL column back to the main dataframe
sf_unified_with_totals = sf_unified.join(total_enrolled, on='SchoolName', how='left')

sf_unified_with_totals.head()


SchoolName,Charter,GR_ALL,GR_TK,GR_KN,GR_01,GR_02,GR_03,GR_04,GR_05,GR_06,GR_07,GR_08,GR_09,GR_10,GR_11,GR_12,Race/Ethnicity,TOTAL_ALL
str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,i64
"""District Office""","""N""",21,21,0,0,0,0,0,0,0,0,0,0,0,0,0,"""Asian""",66
"""District Office""","""N""",1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,"""African American""",66
"""District Office""","""N""",2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,"""Not Reported""",66
"""District Office""","""N""",2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,"""Filipino""",66
"""District Office""","""N""",21,21,0,0,0,0,0,0,0,0,0,0,0,0,0,"""Hispanic or Latino""",66


In [15]:
# compute percentages of each grade column
grade_columns = [f'GR_{str(i).zfill(2)}' for i in range(1, 13)] + ['GR_TK', 'GR_KN', 'GR_ALL']

sf_unified_percentages = sf_unified_with_totals.with_columns([
    (polars.col(col) / polars.col('TOTAL_ALL') * 100).alias(f'{col}_PCT')
    for col in grade_columns
])

# Round percentage columns to two decimal places
percentage_columns = [col for col in sf_unified.columns if col.endswith('_PCT')]
sf_unified_percentages.head()


SchoolName,Charter,GR_ALL,GR_TK,GR_KN,GR_01,GR_02,GR_03,GR_04,GR_05,GR_06,GR_07,GR_08,GR_09,GR_10,GR_11,GR_12,Race/Ethnicity,TOTAL_ALL,GR_01_PCT,GR_02_PCT,GR_03_PCT,GR_04_PCT,GR_05_PCT,GR_06_PCT,GR_07_PCT,GR_08_PCT,GR_09_PCT,GR_10_PCT,GR_11_PCT,GR_12_PCT,GR_TK_PCT,GR_KN_PCT,GR_ALL_PCT
str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""District Office""","""N""",21,21,0,0,0,0,0,0,0,0,0,0,0,0,0,"""Asian""",66,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31.818182,0.0,31.818182
"""District Office""","""N""",1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,"""African American""",66,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.515152,0.0,1.515152
"""District Office""","""N""",2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,"""Not Reported""",66,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.030303,0.0,3.030303
"""District Office""","""N""",2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,"""Filipino""",66,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.030303,0.0,3.030303
"""District Office""","""N""",21,21,0,0,0,0,0,0,0,0,0,0,0,0,0,"""Hispanic or Latino""",66,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31.818182,0.0,31.818182


## Pivot by School

In [21]:
# pivot the dataset by school wtih whole shool demographics as columns
sf_unified_pivoted = sf_unified_percentages.pivot(
    index=['SchoolName'],
    on='Race/Ethnicity',
    values=['GR_ALL', 'GR_ALL_PCT']
)

# Generate the rename mapping dynamically
rename_mapping = {}
for category in reporting_category_map.values():
    # For student count columns
    rename_mapping[f'GR_ALL_{category}'] = f'{category.lower().replace(' ', '_')}_students'
    
    # For percentage columns
    rename_mapping[f'GR_ALL_PCT_{category}'] = f'{category.lower().replace(' ', '_')}_percent'


# Rename the columns
sf_unified_pivoted = sf_unified_pivoted.rename(rename_mapping)


# replace spaces with underscores in column names
sf_unified_pivoted = sf_unified_pivoted\
    .rename({
        col: col.replace(' ', '') for col in sf_unified_pivoted.columns
    }) \
    .rename({
        'SchoolName': 'school_name'
    })


# normalize school names
sf_unified_pivoted = sf_unified_pivoted.with_columns(
    polars.col("school_name").str.replace(
        r"Asawa \(Ruth\) SF Sch of the Arts\, A Public School\b", 
        "Asawa (Ruth) SF Sch of the Arts A Public School"
    )
)

sf_unified_pivoted.write_csv(workspace_path.joinpath('data/processed/race_ethnicity_demographics.csv'))
sf_unified_pivoted


school_name,asian_students,african_american_students,not_reported_students,filipino_students,hispanic_or_latino_students,two_or_more_races_students,white_students,pacific_islander_students,american_indian_or_alaska_native_students,asian_percent,african_american_percent,not_reported_percent,filipino_percent,hispanic_or_latino_percent,two_or_more_races_percent,white_percent,pacific_islander_percent,american_indian_or_alaska_native_percent
str,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""District Office""",21,1,2,2,21,7,12,null,null,31.818182,1.515152,3.030303,3.030303,31.818182,10.606061,18.181818,null,null
"""KIPP Bayview Academy""",3,62,null,null,95,9,null,16,null,1.621622,33.513514,null,null,51.351351,4.864865,null,8.648649,null
"""KIPP San Francisco Bay Academy""",9,55,null,3,178,5,12,4,null,3.383459,20.676692,null,1.12782,66.917293,1.879699,4.511278,1.503759,null
"""Five Keys Charter (SF Sheriff'…",2,21,null,1,112,7,9,1,null,1.30719,13.72549,null,0.653595,73.202614,4.575163,5.882353,0.653595,null
"""Jordan (June) School for Equit…",9,18,6,10,147,9,3,4,null,4.368932,8.737864,2.912621,4.854369,71.359223,4.368932,1.456311,1.941748,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Creative Arts Charter""",18,51,4,2,96,66,192,null,null,4.195804,11.888112,0.932401,0.4662,22.377622,15.384615,44.755245,null,null
"""Yu (Alice Fong) Elementary""",352,33,42,18,29,83,29,null,3,59.762309,5.602716,7.13073,3.056027,4.923599,14.091681,4.923599,null,0.509338
"""Chin (John Yehall) Elementary""",207,2,8,7,11,3,15,null,null,81.818182,0.790514,3.162055,2.766798,4.347826,1.185771,5.928854,null,null
